In [1]:
import pandas as pd
df=pd.read_csv("SPY.USUSD_Candlestick_1_D_BID_16.02.2017-27.04.2024.csv")

In [2]:
df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
#df.set_index("Gmt time", inplace=True)
df=df[df.High!=df.Low]
df.reset_index(inplace=True, drop=True)

In [3]:
import pandas_ta as ta
df['emafast'] = df.ta.ema(length=30)
df['emaslow'] = df.ta.ema(length=60)

In [4]:
def TotalSignal(df, emafast, emaslow):
    df['TotalSignal'] = 0
    for row in range(1, len(df)):
        if (emafast[row-1] <= emaslow[row-1]) and (emafast[row] > emaslow[row]):
            df.at[row, 'TotalSignal'] = 2
        elif (emafast[row-1] >= emaslow[row-1]) and (emafast[row] < emaslow[row]):
            df.at[row, 'TotalSignal'] = 1
    return df
        
TotalSignal(df, emafast=df.emafast, emaslow=df.emaslow)

Gmt time     Open     High      Low    Close     Volume     emafast  \
0    2017-02-16  234.580  234.948  233.850  234.726    58.1260         NaN   
1    2017-02-17  233.945  235.060  233.930  235.045    63.6515         NaN   
2    2017-02-21  235.527  236.679  235.510  236.406    61.4310         NaN   
3    2017-02-22  236.006  236.530  235.830  236.277    57.9701         NaN   
4    2017-02-23  236.879  236.890  235.560  236.438    66.0936         NaN   
...         ...      ...      ...      ...      ...        ...         ...   
1805 2024-04-22  497.816  502.367  495.426  499.656  1299.6000  509.521266   
1806 2024-04-23  501.787  506.076  501.207  505.606  1100.7960  509.268668   
1807 2024-04-24  506.516  507.356  503.116  505.456  1251.6480  509.022689   
1808 2024-04-25  499.077  504.257  497.486  503.377  1590.3840  508.658451   
1809 2024-04-26  506.347  509.867  505.696  508.237  1104.6480  508.631261   

         emaslow  TotalSignal  
0            NaN            0  
1            NaN            0  
2            NaN            0  
3            NaN            0  
4            NaN            0  
...          ...          ...  
1805  503.772662            0  
1806  503.832772            0  
1807  503.885992            0  
1808  503.869304            0  
1809  504.012507            0  

[1810 rows x 9 columns]

In [5]:
import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-3
    elif x['TotalSignal']==2:
        return x['Low']-1e-3
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[500:700]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'],
                increasing=dict(line=dict(color='rgba(0, 255, 0, 0.6)', width=0.1), # Red with transparency for increasing
                fillcolor='rgba(0, 255, 0, 0.6)'),  # Match fill color with line color
                decreasing=dict(line=dict(color='rgba(255, 0, 0, 0.6)', width=0.1), # Green with transparency for decreasing
                fillcolor='rgba(255, 0, 0, 0.6)')), # Match fill color with line color),
                go.Scatter(x=dfpl.index, y=dfpl.emafast, line=dict(color='yellow', width=2), name="EMAFast"),
                go.Scatter(x=dfpl.index, y=dfpl.emaslow, line=dict(color='magenta', width=2), name="EMASlow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="Signal")

fig.update_layout(
    width=1000,
    height=800,
    sliders=[],
    paper_bgcolor='black',  # Set the background color of the entire figure
    plot_bgcolor='black',   # Set the background color of the plotting area
    xaxis_showgrid=False,   # Remove x-axis gridlines
    yaxis_showgrid=False,   # Remove y-axis gridlines
)

fig.show()

In [7]:
dfpl = df[:].copy()

In [8]:
from backtesting import Strategy, Backtest
import pandas_ta as ta

dfpl["EMAFast"] = ta.ema(dfpl["Close"], length=5)
dfpl["EMASlow"] = ta.ema(dfpl["Close"], length=20)

def SIGNAL():
    return dfpl.TotalSignal

class MyStrat(Strategy):
    fast_ema_len = 5
    slow_ema_len = 20
    mysize = 0.1

    def init(self):
        dfpl["EMAFast"] = ta.ema(dfpl["Close"], length=self.fast_ema_len)
        dfpl["EMASlow"] = ta.ema(dfpl["Close"], length=self.slow_ema_len)
        self.fast_ema = self.I(lambda: dfpl["EMAFast"])
        self.slow_ema = self.I(lambda: dfpl["EMASlow"])

    def next(self):
        if self.fast_ema[-1] > self.slow_ema[-1] and self.fast_ema[-2] <= self.slow_ema[-2]:
            for trade in self.trades:
                if trade.is_short:
                    trade.close()
            if len(self.trades) == 0:
                self.buy(size=self.mysize)

        elif self.fast_ema[-1] < self.slow_ema[-1] and self.fast_ema[-2] >= self.slow_ema[-2]:
            for trade in self.trades:
                if trade.is_long:
                    trade.close()
            if len(self.trades) == 0:
                self.sell(size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/10, commission=.000)
stats, heatmap = bt.optimize(fast_ema_len=range(5, 100, 5),
                    slow_ema_len=range(40, 200, 10),
                    maximize='Return [%]', max_tries=400,
                    random_state=0,
                    return_heatmap=True)
stats

C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_3280\2044448718.py:36: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.

c:\Users\Serkan POLAT\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning:

Searching for best of 304 configurations.



  0%|          | 0/16 [00:00<?, ?it/s]

Start                                     0.0
End                                    1809.0
Duration                               1809.0
Exposure Time [%]                   71.712707
Equity Final [$]                   190398.491
Equity Peak [$]                    197117.235
Return [%]                          90.398491
Buy & Hold Return [%]              116.523521
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -14.215756
Avg. Drawdown [%]                   -1.404031
Max. Drawdown Duration                  586.0
Avg. Drawdown Duration              19.383721
# Trades                                 27.0
Win Rate [%]                        55.555556
Best Trade [%]                       24.77333
Worst Trade [%]                      -4.46545
Avg. Trade [%]                    

In [9]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Convert multiindex series to dataframe
heatmap_df = heatmap.unstack()
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_df, annot=True, cmap='viridis', fmt='.0f')
plt.show()

In [10]:
bt.plot()

Row(id='1427', ...)